In [22]:
import fermionic_mbody as fmb
import numpy as np
import openfermion as of

n, m = 2, 1
d, num, pairs = 2*n, 2*m, False
basis = fmb.FixedBasis(d=d, num=num, pairs=pairs)
rho_1_arrays = fmb.rho_m_gen(basis, 1)
rho_2_arrays = fmb.rho_m_gen(basis, 2)
rho_2_kkbar_arrays = fmb.rho_2_kkbar_gen(basis)
rho_2_block_arrays = fmb.rho_2_block_gen(basis)
rho_2_block_arrays.shape

ρ₂-block: 100%|██████████| 8/8 [00:00<00:00, 1355.30it/s]


(4, 4, 6, 6)

In [23]:
def pair_condensate_state(basis: fmb.FixedBasis, m_pairs: int) -> np.ndarray:
    d = basis.d
    assert d % 2 == 0
    n_pairs = d // 2
    assert 0 < m_pairs <= n_pairs

    vec = np.zeros(basis.size, complex)

    # iterate over every subset of m time-reversed pairs
    from itertools import combinations

    for combo in combinations(range(n_pairs), m_pairs):
        bitmask = 0
        for k in combo:
            bitmask |= 1 << (2 * k)      # c†_k
            bitmask |= 1 << (2 * k + 1)  # c†_{k̄}
        idx = int(np.where(basis.num_ele == bitmask)[0])
        vec[idx] = 1.0

    vec /= np.linalg.norm(vec)
    return vec.real

def a_dag_state(basis: fmb.FixedBasis, sk_arr: np.ndarray):
    # Generamos los operadores A^\dag
    op = of.FermionOperator.zero()
    for k in range(0, basis.d//2):
        op += sk_arr[k] * of.FermionOperator((2*k, 1)) * of.FermionOperator((2*k+1, 1))
    op = np.prod([op for _ in range(basis.m//2)])
    vect = basis.opr_to_vect(op).real
    print(op)
    return vect * 1/np.linalg.norm(vect)

def partial_traspose(rho, perm):
    d = basis.d//2
    rho_p = np.zeros((d**2, d**2))
    for m in range(d**2):
        for n in range(d**2):
            # Escrimos los índices i j k l en términos de m n
            i, j = m//d, m % d
            k, l = n//d, n % d
            indices = (i, j, k, l)
            ip, jp, kp, lp = tuple(indices[p] for p in perm)
            rho_p[m,n] = rho[ip*d + jp, kp*d + lp]
    return rho_p

def slater_state(basis: fmb.FixedBasis, occupied) -> np.ndarray:
    bitmask = sum(1 << i for i in occupied)
    idx = int(np.where(basis.num_ele == bitmask)[0])
    vec = np.zeros(basis.size, complex)
    vec[idx] = 1.0
    return vec

In [24]:
# Uniforme
sk, lam = 1/np.sqrt(basis.d//2) * np.ones(basis.d//2), -1*m*(n+1-2*m)/(n*(n-1))

vect = a_dag_state(basis, sk)
#vect = slater_state(basis, np.random.randint(0, basis.d, basis.m))

rho = fmb.rho_m(vect, rho_2_block_arrays)

rho_pt = partial_traspose(rho, [0, 3, 2, 1]) - partial_traspose(rho, [0, 2, 3, 1])
au = lambda x: np.sort(np.linalg.eigvals(x).real)
au(rho_pt), lam

0.7071067811865475 [0^ 1^] +
0.7071067811865475 [2^ 3^]


(array([-1.,  0.,  0.,  0.]), -0.5)